# ETL file to convert the geojson file into network files

In [5]:
import json 
import csv
import networkx as nx
import os, sys

In [6]:
nodefile = "vertex/vertex.csv"
edgefile = "edge/edge.csv"
jsonfile = "geojson/multigraph.geojson"
graphmlfile = "graphml/multigraph.graphml"
gmlfile = "gml/multigraph.gml"

if not os.path.exists("edge"):
    os.makedirs("edge")
if not os.path.exists("vertex"):
    os.makedirs("vertex")
if not os.path.exists("gml"):
    os.makedirs("gml")

In [7]:
with open(jsonfile, 'r') as jsfile:
    with open(nodefile, 'w+') as nodefiled:
        with open(edgefile, 'w+') as edgefiled:
            node = csv.writer(nodefiled)
            edge = csv.writer(edgefiled)
            # Header
            node.writerow(["# NodeID", "Lat", " Lon", "Layer"])
            edge.writerow(["# EdgeID", "Source NodeID", "Target NodeID", "Direction", "Layer"])
            for line in jsfile:
                jsentry = json.loads(line)
                if jsentry['properties']['type'] == "node":
                    node.writerow([
                            jsentry['_id']['$oid'], 
                            jsentry['geometry']['coordinates'][0], 
                            jsentry['geometry']['coordinates'][1], 
                            jsentry['properties']['layer']])
                if jsentry['properties']['type'] == "edge":
                    if jsentry['properties']['name'].startswith("54"):
                        jsentry['properties']['name'] = "None"
                    if 'direction' in jsentry['properties']:
                        if jsentry['properties']['direction'] == "Double sens":
                            direction = "TwoWay"
                        elif (jsentry['properties']['direction'] == "Sens inverse" 
                              or jsentry['properties']['direction'] == "Sens unique"):
                            direction = "OneWay"
                            
                        edge.writerow([
                            jsentry['properties']['mongo_org_id'],
                            jsentry['properties']['mongo_dest_id'],
                            jsentry['_id']['$oid'], 
                            direction,
                            jsentry['properties']['layer'], 
                            jsentry['properties']['name']])
                    else:
                        edge.writerow([
                            jsentry['properties']['mongo_org_id'],
                            jsentry['properties']['mongo_dest_id'],
                            jsentry['_id']['$oid'],
                            "TwoWay",
                            jsentry['properties']['layer'], 
                            jsentry['properties']['name']])

In [8]:
G = nx.DiGraph()

with open(nodefile, 'r') as node:
    reader = csv.reader(node)
    next(reader)
    for row in reader:
        if len(row) != 0:
            lat = row[1]
            lon = row[2]
            ntype = row[3]
            G.add_node(row[0], lat=lat, lon=lon, type=ntype)

with open(edgefile, 'r') as node:
    reader = csv.reader(node)
    next(reader)
    for row in reader:
        if len(row) != 0:
            G.add_edge(row[1], row[2], type=row[4], name=row[5])
            if row[3] == 'TwoWay':
                G.add_edge(row[2], row[1], type=row[4], name=row[5])
        
nx.write_graphml(G, graphmlfile)
nx.write_graphml(G, gmlfile)

In [9]:
print(G.number_of_nodes())
print(G.number_of_edges())

42678
51931


In [1]:
try:
    from folium import *
    from folium import plugins
except:
    %pip install folium

In [2]:
import pandas as pd

df = pd.read_json('geojson/multigraph.geojson', lines=True)

id = pd.json_normalize(df["_id"])
id = id.rename(columns={"$oid": "ID"})

geom = pd.json_normalize(df["geometry"])
geom = geom.rename(columns={"type": "LP"})

prop = pd.json_normalize(df["properties"])

In [3]:
new_df = pd.concat([id, geom, prop], axis=1)
new_df

,ID,LP,coordinates,database_ref,layer,type,database_id,database_org_id,mongo_dest_id,name,db_id,database_dest_id,direction,mongo_org_id,length,edge_type,degreeDual
0,5453b63355474a3362317270,Point,"[3.537235554510594, 48.231939820695146]",IGN,road,node,73469,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,5453b63355474a3362317271,Point,"[2.395553960951333, 49.049663371625094]",IGN,road,node,35541,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,5453b63355474a3362317272,Point,"[1.65630447868506, 49.05187906283865]",IGN,road,node,35234,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,5453b63355474a3362317273,Point,"[2.091532960687826, 49.05588679019185]",IGN,road,node,35235,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5453b63355474a3362317274,Point,"[2.146181135828161, 49.05648991168742]",IGN,road,node,35231,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42673,54b954ba55474a0e08bd3f33,LineString,"[[2.3322643, 48.8707693], [2.3304290066, 48.87...",NaN,crosslayer,edge,NaN,NaN,5453b63555474a336231a8c5,54b954ba55474a0e08bd3f33,NaN,NaN,twoway,5453b63455474a336231911d,NaN,NaN,45.0
42674,54b954ba55474a0e08bd3f34,LineString,"[[2.3144204, 48.893964], [2.3143569, 48.8939663]]",NaN,crosslayer,edge,NaN,NaN,5453b63455474a336231821b,54b954ba55474a0e08bd3f34,NaN,NaN,twoway,5453b63555474a336231a233,NaN,NaN,21.0
42675,54b954ba55474a0e08bd3f35,LineString,"[[2.3301258, 48.8313954], [2.3331828, 48.83309...",NaN,crosslayer,edge,NaN,NaN,5453b63455474a33623177e3,54b954ba55474a0e08bd3f35,NaN,NaN,twoway,5453b63555474a336231aeba,NaN,NaN,38.0
42676,54b954ba55474a0e08bd3f36,LineString,"[[2.3443622, 48.8535657], [2.345551, 48.8534338]]",NaN,crosslayer,edge,NaN,NaN,5453b63555474a336231a5ca,54b954ba55474a0e08bd3f36,NaN,NaN,twoway,5453b63555474a336231a416,NaN,NaN,87.0


In [6]:
only_point = new_df.loc[new_df['LP'] == "Point"]
only_point.loc[only_point['layer']=="tram"]
only_point.layer.unique().tolist()

['road', 'train', 'metro', 'tram']

In [11]:
from math import sin, cos, sqrt, atan2, radians
#layers = only_point.layer.unique().tolist()
#layerTypes = only_point.layer.tolist()
#points = only_point.coordinates.tolist()

class MapMaker:
    def __init__(self, df):
        self.df = df
        self.only_point = df.loc[new_df['LP'] == "Point"]
        self.points = self.only_point.coordinates.tolist()
        self.center = (48.85654066902656, 2.349154275836)
        self.canvas = True,
        self.bounds = True,
        self.filename = "france.html"
        self.layerColor = {
            "road": "green",
            "train": "blue",
            "metro": "gray",
            "tram": "purple"
        }
    
    @staticmethod
    def clear():
        if os.name == 'posix':
            os.system('clear')
        else:
            os.system('cls')

    @staticmethod
    def calculate_distance(lon1, lat1, lon2, lat2, R=6373.0):
        lat1 = radians(lat1)
        lat2 = radians(lat2)
        lon1 = radians(lon1)
        lon2 = radians(lon2)

        dlon = lon2 - lon1
        dlat = lat2 - lat1

        a = sin(dlat / 2) ** 2 + cos(lat1) * cos(lat2) * sin(dlon / 2) ** 2
        c = 2 * atan2(sqrt(a), sqrt(1 - a))

        distance = R * c
        return distance
      
    def create_map(self):
        if not os.path.exists(self.filename):
            m = Map(location=[self.center[0], self.center[1]], 
                tiles="CartoDB positron", 
                min_zoom=7, 
                zoom_start=9,
                zoom_control=True, 
                min_lat=42, 
                max_lat=54, 
                min_lon=-10, 
                max_lon=14, 
                max_bounds=self.bounds,
                prefer_canvas=self.canvas)

            minimap = plugins.MiniMap()
            m.add_child(minimap)

            layers = self.only_point.layer.unique().tolist()

            fg = FeatureGroup(control=False, show=False)
            m.add_child(fg)

            f1 = plugins.FeatureGroupSubGroup(fg, layers[0].capitalize())
            m.add_child(f1)

            f2 = plugins.FeatureGroupSubGroup(fg, layers[1].capitalize())
            m.add_child(f2)

            f3 = plugins.FeatureGroupSubGroup(fg, layers[2].capitalize())
            m.add_child(f3)

            f4 = plugins.FeatureGroupSubGroup(fg, layers[3].capitalize())
            m.add_child(f4)

            i = 0
            layerTypes = self.only_point.layer.tolist()

            for lon, lat in self.points:

                distance = self.calculate_distance(lon, lat, self.center[1], self.center[0])
                radiusDistance = distance

                if int(distance) > 0:
                    distance = int(distance)
                    distance = f"{distance} km"
                else:
                    distance = distance * 1000
                    distance = int(distance)
                    distance = f"{distance} mt"

                circle = Circle(
                    location=(lat, lon),
                    tooltip=f"<strong>Type:</strong> {layerTypes[i]}<br><strong>Center distance:</strong> {distance}",
                    radius=sqrt(radiusDistance * 1000),
                    popup=f"<strong>Type:</strong> {layerTypes[i]}<br><strong>Lat:</strong> {lat}<br><strong>Long:</strong> {lon}<br><strong>Center distance:</strong> {distance}",
                    color=self.layerColor.get(layerTypes[i]),
                    fill=False,
                    fill_color=self.layerColor.get(layerTypes[i])
                )
                
                if layerTypes[i] == layers[0]:
                    f1.add_child(circle)
                elif layerTypes[i] == layers[1]:
                    f2.add_child(circle)
                elif layerTypes[i] == layers[2]:
                    f3.add_child(circle)
                elif layerTypes[i] == layers[3]:
                    f4.add_child(circle)

                i +=1

            paris = Circle(
                location=(self.center[0], self.center[1]),
                tooltip="Center",
                popup="",
                radius=80,
                color="crimson",
                fill=True,
                fill_color="crimson"
            )
            paris.add_to(fg)

            plugins.Fullscreen(
                    position="topright",
                    title="Fullscreen",
                    title_cancel="Exit fullscreen",
                    force_separate_button=True,
                ).add_to(m)

            LayerControl().add_to(m)
            m.save(self.filename)

In [12]:
MapMaker(new_df).create_map()